In [46]:
# set cwd and solver
import os
import cobra
cobra.Configuration().solver = "gurobi"
os.chdir('C:/Users/prins/GitHub/Human1_RPE-PR') 

In [47]:
# load all models in \cs_mods\July2023 folder by cycling through names
from cobra.io import read_sbml_model    
import glob 
import os   
from pathlib import Path
import pandas as pd

# do not change directory but still go to the folder with the models
folder = Path().cwd() / "cs_mods" / "July2023"
model_list = []
for file in folder.glob("*.xml"):  
    model = read_sbml_model(file)
    model.id = file.stem # remove .xml 
    model.name = file.stem # remove .xml
    model_list.append(model)

No objective coefficients in model. Unclear what should be optimized
No objective coefficients in model. Unclear what should be optimized
No objective coefficients in model. Unclear what should be optimized
No objective coefficients in model. Unclear what should be optimized


In [48]:
# load old combined RPE_PR models
mod_RPE_PR = read_sbml_model(Path().cwd() / "models" / "mod_RPE_PR.xml")
mod_RPE_PR.id = 'RPE_PR_old'
mod_RPE_PR.name = 'RPE_PR_old'
mod_Human1_Human1 = read_sbml_model(Path().cwd() / "models" /  "mod_Human1_Human1.xml")
mod_Human1_Human1.id = 'RPE_PR_control_old'
mod_Human1_Human1.name = 'RPE_PR_control_old'
mod = read_sbml_model(Path().cwd() / 'models/Human-GEM.xml')
mod.id = 'Human1_old'
mod.name = 'Human1_old'

# add old models to model_list
model_list = model_list + [mod,mod_RPE_PR]

# check out model_list
model_list

[<Model Human-GEM_28062023 at 0x1b3287bd7b0>,
 <Model mod_RPE_PRcone_Liang at 0x1b32dbdb1f0>,
 <Model mod_RPE_PRcone_Liang_singlePR at 0x1b3352ebe80>,
 <Model mod_RPE_PRcone_Liang_singleRPE at 0x1b33884afb0>,
 <Model mod_RPE_PRcone_Lukowski at 0x1b33d28c070>,
 <Model mod_RPE_PRcone_Lukowski_singlePR at 0x1b344bf7700>,
 <Model mod_RPE_PRcone_Lukowski_singleRPE at 0x1b348a87850>,
 <Model mod_RPE_PRcontrol at 0x1b34cbd5030>,
 <Model mod_RPE_PRrod_Liang at 0x1b3626d6920>,
 <Model mod_RPE_PRrod_Liang_singlePR at 0x1b3761da650>,
 <Model mod_RPE_PRrod_Liang_singleRPE at 0x1b3784ae6b0>,
 <Model mod_RPE_PRrod_Lukowski at 0x1b37c5c9f30>,
 <Model mod_RPE_PRrod_Lukowski_singlePR at 0x1b385af6560>,
 <Model mod_RPE_PRrod_Lukowski_singleRPE at 0x1b387f27cd0>,
 <Model Human1_old at 0x1b3a8c3f730>,
 <Model RPE_PR_old at 0x1b32833df60>]

In [49]:
# modify models
# load blood exchange bounds
df = pd.read_excel(Path().cwd() / 'rxn_bounds/blood_selection.xlsx') # with col names 'id' 'lb' 'ub'

# for all models in the list set objective function (ATP hydrolysis, in PR for combined models)
for m in model_list:
    if 'MAR03964_PR' in [r.id for r in m.reactions]:
        m.objective = 'MAR03964_PR'
    elif 'MAR03964_RPE' in [r.id for r in m.reactions]:
        m.objective = 'MAR03964_RPE'
    elif 'MAR03964' in [r.id for r in m.reactions]:
        m.objective = 'MAR03964'
    else:
        print('no ATP hydrolysis reaction (MAR03964) in model: ' + m.id)

# list combined and single models
combined_models = [m for m in model_list if 'single' not in m.id and 'Human' not in m.id]
single_models = [m for m in model_list if 'single' in m.id]
Human1_models = [m for m in model_list if 'Human' in m.id]

# close PR exchange in combined models and open RPE exchange (efflux)
for m in combined_models:
    for r in [r for r in m.reactions if len(r.products) == 0 ]:           # close all exchange reactions
        r.bounds=(0,0)
    for r in [r for r in m.reactions if len(r.products) == 0 if '_RPE' in r.id]:           # open efflux for RPE exchange reactions
        r.bounds=(0,1000) 

# only allow efflux exchange in single models
for m in single_models:
    for r in [r for r in m.reactions if len(r.products) == 0 ]:           # close all exchange reactions
        r.bounds=(0,1000)

# for Human1 models, open efflux
for m in [m for m in model_list if 'Human' in m.id]:
    for r in [r for r in m.reactions if len(r.products) == 0]:           # open efflux for RPE exchange reactions
        r.bounds=(0,1000)

In [50]:
def set_exchange_bounds(model, ex_dict): 
    """
    set bounds for exchange reactions in model based on dict

    Parameters
    ----------
    model : cobra.Model
        model to set bounds for
    ex_dict : dict
        dictionary with exchange reaction ids as keys and bounds as values

    Returns
    -------
    model : cobra.Model
        model with updated bounds

    """

    for ex in ex_dict.keys():
        if ex + '_RPE' in [r.id for r in model.reactions]:
            model.reactions.get_by_id(ex + '_RPE').bounds = ex_dict[ex] # set bounds for RPE exchange
        elif ex + '_PR' in [r.id for r in model.reactions]:
            model.reactions.get_by_id(ex + '_PR').bounds = ex_dict[ex] # if no RPE exchange, set bounds for PR exchange
        elif ex + '_eRPE_PR' in [r.id for r in model.reactions]:
            model.reactions.get_by_id(ex + '_eRPE_PR').bounds = ex_dict[ex] # if no RPE or PR exchange, set bounds for eRPE_PR exchange
        elif ex in [r.id for r in model.reactions]:
            model.reactions.get_by_id(ex).bounds = ex_dict[ex] # if no RPE, PR, or eRPE_PR exchange, set bounds for exchange (generic)
        else:  
            print('no exchange reaction for ' + ex + ' in model: ' + model.id) # if no exchange reaction, print message
    return model 



ex_bounds = {'MAR09048':(-3.51,1000),'MAR09034':(-4.18,1000),'MAR09135':(7.25,1000)}


results_dict = {}
for m in model_list:
    set_exchange_bounds(m, ex_bounds)
    m.optimize()
    results_dict[m.id] = m.objective.value
df1 = pd.DataFrame(results_dict, index = ['max ATP hydrolysis (pmol/s/mm^2)']).T
df1

,max ATP hydrolysis (pmol/s/mm^2)
Human-GEM_28062023,24.7325
mod_RPE_PRcone_Liang,23.9000
mod_RPE_PRcone_Liang_singlePR,11.1350
mod_RPE_PRcone_Liang_singleRPE,23.9000
mod_RPE_PRcone_Lukowski,24.7325
mod_RPE_PRcone_Lukowski_singlePR,22.2350
mod_RPE_PRcone_Lukowski_singleRPE,23.9000
mod_RPE_PRcontrol,24.7325
mod_RPE_PRrod_Liang,23.9000
mod_RPE_PRrod_Liang_singlePR,13.9100


In [51]:
ex_bounds = {'MAR09048':(-3.51,1000),'MAR09034':(-4.18,1000),'MAR09135':(7.25,1000)}

def fba_(model_list, ex_bounds, sweep_list, sweep_reaction):

    df3 = pd.DataFrame()
    for i in sweep_list:
        results_dict = {}
        for m in model_list:
            set_exchange_bounds(m, ex_bounds)
            set_exchange_bounds(m, {sweep_reaction:(-i,1000)})
            m.optimize()
            results_dict[m.id] = m.objective.value
        df2 = pd.DataFrame(results_dict, index = [str(i) + \
            ' pmol/s/mm^2 influx ' + str(sweep_reaction) +\
            ' --- max ATP hydrolysis (pmol/s/mm^2)']).T
        # put all results in one df using concat    
        df3 = pd.concat([df3,df2],axis=1)

    return df3

sweep_list = [1,2,3,4,4.18,5,6,7,8,9, 10, 100, 1000]
sweep_reaction = 'MAR09034'
ex_bounds = {'MAR09048':(-3.51,1000),'MAR09034':(-4.18,1000),'MAR09135':(7.25,1000)}
df3 = fba_(model_list, ex_bounds, sweep_list, sweep_reaction)
df3


c:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
c:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
c:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
c:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
c:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
c:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\u

,1 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),2 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),3 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),4 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),4.18 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),5 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),6 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),7 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),8 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),9 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),10 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),100 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),1000 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2)
Human-GEM_28062023,None,None,None,19.0625,24.7325,27.455000,29.550000,31.550000,33.550000,35.550000,37.550000,217.550000,1000.0
mod_RPE_PRcone_Liang,None,None,None,18.5000,23.9000,27.170000,29.550000,31.550000,33.550000,35.550000,37.550000,217.550000,1000.0
mod_RPE_PRcone_Liang_singlePR,None,None,None,9.8750,11.1350,16.875000,23.875000,30.875000,33.550000,35.550000,37.550000,217.550000,1000.0
mod_RPE_PRcone_Liang_singleRPE,None,None,None,18.5000,23.9000,27.170000,29.550000,31.550000,33.550000,35.550000,37.550000,217.550000,1000.0
mod_RPE_PRcone_Lukowski,None,None,None,19.0625,24.7325,27.455000,29.550000,31.550000,33.550000,35.550000,37.550000,217.550000,1000.0
mod_RPE_PRcone_Lukowski_singlePR,None,None,None,17.3750,22.2350,26.600000,29.550000,31.550000,33.550000,35.550000,37.550000,217.550000,1000.0
mod_RPE_PRcone_Lukowski_singleRPE,None,None,None,18.5000,23.9000,27.170000,29.550000,31.550000,33.550000,35.550000,37.550000,217.550000,1000.0
mod_RPE_PRcontrol,None,None,None,19.0625,24.7325,27.455000,29.550000,31.550000,33.550000,35.550000,37.550000,217.550000,1000.0
mod_RPE_PRrod_Liang,None,None,None,18.5000,23.9000,27.170000,29.550000,31.550000,33.550000,35.550000,37.550000,217.550000,1000.0
mod_RPE_PRrod_Liang_singlePR,None,None,None,11.7500,13.9100,23.750000,29.550000,31.550000,33.550000,35.550000,37.550000,217.550000,1000.0


In [52]:
sweep_list = [1,2,3,4,4.18,5,6,7,8,9, 10, 100, 1000]
sweep_reaction = 'MAR09034'
ex_bounds = {'MAR09048':(-3.51,1000),'MAR09034':(-4.18,1000),'MAR09135':(0,1000)}
df4 = fba_(model_list, ex_bounds, sweep_list, sweep_reaction)
df4

,1 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),2 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),3 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),4 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),4.18 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),5 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),6 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),7 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),8 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),9 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),10 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),100 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),1000 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2)
Human-GEM_28062023,19.361250,21.550000,23.550000,25.550000,25.910000,27.550000,29.550000,31.550000,33.550000,35.550000,37.550000,217.550000,1000.0
mod_RPE_PRcone_Liang,18.795000,21.550000,23.550000,25.550000,25.910000,27.550000,29.550000,31.550000,33.550000,35.550000,37.550000,217.550000,1000.0
mod_RPE_PRcone_Liang_singlePR,7.000000,14.000000,21.000000,25.550000,25.910000,27.550000,29.550000,31.550000,33.550000,35.550000,37.550000,217.550000,1000.0
mod_RPE_PRcone_Liang_singleRPE,18.795000,21.550000,23.550000,25.550000,25.910000,27.550000,29.550000,31.550000,33.550000,35.550000,37.550000,217.550000,1000.0
mod_RPE_PRcone_Lukowski,19.361250,21.550000,23.550000,25.550000,25.910000,27.550000,29.550000,31.550000,33.550000,35.550000,37.550000,217.550000,1000.0
mod_RPE_PRcone_Lukowski_singlePR,17.662500,21.550000,23.550000,25.550000,25.910000,27.550000,29.550000,31.550000,33.550000,35.550000,37.550000,217.550000,1000.0
mod_RPE_PRcone_Lukowski_singleRPE,18.795000,21.550000,23.550000,25.550000,25.910000,27.550000,29.550000,31.550000,33.550000,35.550000,37.550000,217.550000,1000.0
mod_RPE_PRcontrol,19.361250,21.550000,23.550000,25.550000,25.910000,27.550000,29.550000,31.550000,33.550000,35.550000,37.550000,217.550000,1000.0
mod_RPE_PRrod_Liang,18.795000,21.550000,23.550000,25.550000,25.910000,27.550000,29.550000,31.550000,33.550000,35.550000,37.550000,217.550000,1000.0
mod_RPE_PRrod_Liang_singlePR,12.000000,21.550000,23.550000,25.550000,25.910000,27.550000,29.550000,31.550000,33.550000,35.550000,37.550000,217.550000,1000.0


In [53]:
sweep_list = [1,2,3,4,4.18,5,6,7,8,9, 10, 100, 1000]
sweep_reaction = 'MAR09034'
ex_bounds = {'MAR09048':(0,1000),'MAR09034':(-4.18,1000),'MAR09135':(0,1000)}
df5 = fba_(model_list, ex_bounds, sweep_list, sweep_reaction)
df5

,1 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),2 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),3 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),4 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),4.18 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),5 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),6 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),7 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),8 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),9 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),10 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),100 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),1000 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2)
Human-GEM_28062023,2.0,4.0,6.0,8.0,8.36,10.0,12.0,14.0,16.0,18.0,20.0,200.0,1000.0
mod_RPE_PRcone_Liang,2.0,4.0,6.0,8.0,8.36,10.0,12.0,14.0,16.0,18.0,20.0,200.0,1000.0
mod_RPE_PRcone_Liang_singlePR,2.0,4.0,6.0,8.0,8.36,10.0,12.0,14.0,16.0,18.0,20.0,200.0,1000.0
mod_RPE_PRcone_Liang_singleRPE,2.0,4.0,6.0,8.0,8.36,10.0,12.0,14.0,16.0,18.0,20.0,200.0,1000.0
mod_RPE_PRcone_Lukowski,2.0,4.0,6.0,8.0,8.36,10.0,12.0,14.0,16.0,18.0,20.0,200.0,1000.0
mod_RPE_PRcone_Lukowski_singlePR,2.0,4.0,6.0,8.0,8.36,10.0,12.0,14.0,16.0,18.0,20.0,200.0,1000.0
mod_RPE_PRcone_Lukowski_singleRPE,2.0,4.0,6.0,8.0,8.36,10.0,12.0,14.0,16.0,18.0,20.0,200.0,1000.0
mod_RPE_PRcontrol,2.0,4.0,6.0,8.0,8.36,10.0,12.0,14.0,16.0,18.0,20.0,200.0,1000.0
mod_RPE_PRrod_Liang,2.0,4.0,6.0,8.0,8.36,10.0,12.0,14.0,16.0,18.0,20.0,200.0,1000.0
mod_RPE_PRrod_Liang_singlePR,2.0,4.0,6.0,8.0,8.36,10.0,12.0,14.0,16.0,18.0,20.0,200.0,1000.0


In [54]:
sweep_list = [1,2,3,4,4.18,5,6,7,8,9, 10, 100, 1000]
sweep_reaction = 'MAR09034'
ex_bounds = {'MAR09048':(-1000,1000),'MAR09034':(-4.18,1000),'MAR09135':(0,1000)}
df6 = fba_(model_list, ex_bounds, sweep_list, sweep_reaction)
df6

,1 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),2 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),3 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),4 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),4.18 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),5 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),6 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),7 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),8 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),9 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),10 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),100 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),1000 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2)
Human-GEM_28062023,31.50,63.0,94.50,126.0,131.670,157.50,189.0,220.50,252.0,283.50,315.0,1000.0,1000.0
mod_RPE_PRcone_Liang,29.75,59.5,89.25,119.0,124.355,148.75,178.5,208.25,238.0,267.75,297.5,1000.0,1000.0
mod_RPE_PRcone_Liang_singlePR,7.00,14.0,21.00,28.0,29.260,35.00,42.0,49.00,56.0,63.00,70.0,700.0,1000.0
mod_RPE_PRcone_Liang_singleRPE,29.00,58.0,87.00,116.0,121.220,145.00,174.0,203.00,232.0,261.00,290.0,1000.0,1000.0
mod_RPE_PRcone_Lukowski,31.50,63.0,94.50,126.0,131.670,157.50,189.0,220.50,252.0,283.50,315.0,1000.0,1000.0
mod_RPE_PRcone_Lukowski_singlePR,26.00,52.0,78.00,104.0,108.680,130.00,156.0,182.00,208.0,234.00,260.0,1000.0,1000.0
mod_RPE_PRcone_Lukowski_singleRPE,29.00,58.0,87.00,116.0,121.220,145.00,174.0,203.00,232.0,261.00,290.0,1000.0,1000.0
mod_RPE_PRcontrol,31.50,63.0,94.50,126.0,131.670,157.50,189.0,220.50,252.0,283.50,315.0,1000.0,1000.0
mod_RPE_PRrod_Liang,29.75,59.5,89.25,119.0,124.355,148.75,178.5,208.25,238.0,267.75,297.5,1000.0,1000.0
mod_RPE_PRrod_Liang_singlePR,12.00,24.0,36.00,48.0,50.160,60.00,72.0,84.00,96.0,108.00,120.0,1000.0,1000.0


In [55]:
sweep_list = [1,2,3,4,4.18,5,6,7,8,9, 10, 100, 1000]
sweep_reaction = 'MAR09034'
ex_bounds = {'MAR09048':(-1000,1000),'MAR09034':(-4.18,1000),'MAR09135':(-1000,1000)}
df7 = fba_(model_list, ex_bounds, sweep_list, sweep_reaction)
df7

,1 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),2 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),3 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),4 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),4.18 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),5 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),6 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),7 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),8 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),9 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),10 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),100 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2),1000 pmol/s/mm^2 influx MAR09034 --- max ATP hydrolysis (pmol/s/mm^2)
Human-GEM_28062023,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
mod_RPE_PRcone_Liang,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
mod_RPE_PRcone_Liang_singlePR,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
mod_RPE_PRcone_Liang_singleRPE,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
mod_RPE_PRcone_Lukowski,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
mod_RPE_PRcone_Lukowski_singlePR,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
mod_RPE_PRcone_Lukowski_singleRPE,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
mod_RPE_PRcontrol,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
mod_RPE_PRrod_Liang,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
mod_RPE_PRrod_Liang_singlePR,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0


In [57]:
# write dfs to excel
with pd.ExcelWriter('glucose_sweep_max_ATP.xlsx') as writer:
    # spreadsheet 1
    df1.to_excel(writer, sheet_name='max ATP hydrolysis (ox > -3.51, lac > 7.25, glc > -4.18)')
    # spreadsheet 2
    df3.to_excel(writer, sheet_name='ox > -3.51, lac > 7.25')
    # spreadsheet 3
    df4.to_excel(writer, sheet_name='ox > -3.51, lac > 0')
    # spreadsheet 4
    df5.to_excel(writer, sheet_name='ox > 0, lac > 0')
    # spreadsheet 5
    df6.to_excel(writer, sheet_name='ox > -1000, lac > 0')
    # spreadsheet 6
    df7.to_excel(writer, sheet_name='ox > -1000, lac > -1000')


c:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [ ]:
met_list = list(df['id_single'])



['MAR09034',
 'MAR09063',
 'MAR09061',
 'MAR09067',
 'MAR09041',
 'MAR09046',
 'MAR09068',
 'MAR09044',
 'MAR09069',
 'MAR09040',
 'MAR09038',
 'MAR09066',
 'MAR09064',
 'MAR09043',
 'MAR09065',
 'MAR09045',
 'MAR09071',
 'MAR09062',
 'MAR09042',
 'MAR09070',
 'MAR11961',
 'MAR09039']

In [ ]:
ex_bounds = [{'MAR09048':(-0,1000)}\
            ,{'MAR09048':(-3.51,1000)}\
            ,{'MAR09048':(-1000,1000)}]

df_list = []

for dicts in ex_bounds:
    for m in model_list:
        results_dict = {}
        set_exchange_bounds(m, dicts)
        m.optimize()
        results_dict[m.id] = m.objective.value
    df1 = pd.DataFrame(results_dict, index = ['max ATP hydrolysis (pmol/s/mm^2)']).T
    #append dfs in df_list
    df_list.append(df1)


df_list

[            max ATP hydrolysis (pmol/s/mm^2)
 RPE_PR_old                               2.0,
             max ATP hydrolysis (pmol/s/mm^2)
 RPE_PR_old                         19.621827,
             max ATP hydrolysis (pmol/s/mm^2)
 RPE_PR_old                              31.5]

In [ ]:
# ATP yield per metabolite (flux of 1 for each met)
def ATP_yield_analysis_per_aa(models, bounds_dict):
    from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
    results_dict = dict()
    for m in models:
        for m in models:
            rxns = [r for r in m.reactions if len(r.products) == 0]
            for r in rxns:
                r.bounds=(0,1000) # close all exchange reactions
            for index, row in df.iterrows():  
                if row.id_single in [r.id for r in m.reactions]:
                    m.reactions.get_by_id(row['id_single']).bounds = (-1,1000) # open reaction
                    m.reactions.get_by_id('MAR09048').bounds = oxygen_bounds # OXYGEN
                    fba = m.optimize()
                    results_dict[row['id_single']] = fba.objective_value
                m.reactions.get_by_id(row['id_single']).bounds = (0,1000) # close reaction again
    return results_dict

r1 = ATP_yield_analysis_per_aa(mod,{'MAR09048':(0,1000)}) # anearobic
r2 = ATP_yield_analysis_per_aa(mod,{'MAR09048':(-2.2,1000)}) # limited oxygen
r3 = ATP_yield_analysis_per_aa(mod,{'MAR09048':(-1000,1000)}) # unlimited oxyen

df_results = pd.DataFrame([r1.keys(),r1.values(),r2.values(),r3.values()]).transpose()
df_results.to_clipboard()
df_results

,0,1,2,3
0,MAR09034,2.0,13.138325,31.5
1,MAR09048,0.0,0.0,0.0
2,MAR09135,0.0,11.013325,14.75
3,MAR09047,0.0,0.0,0.0
4,MAR09063,0.522727,11.182626,22.0
5,MAR09061,0.0,11.013325,14.75
6,MAR09067,0.090909,7.75,7.75
7,MAR09041,0.0,10.717467,33.25
8,MAR09046,0.0,10.694739,28.5
9,MAR09068,0.0,10.558376,26.0


In [ ]:
def fba_ATP_rxns(models, boundary_dicts, rxns, boundary_labels=dict(),results_name=''):    
    # inputs: model, list of boundary dicts {rxnID:(lb,ub)}, objective function rxnID (string)
    import pandas as pd
    from datetime import datetime
    from src.get_info import make_rxn_df, make_compact_rxn_df, add_compartment2rxn
    from datetime import datetime

    for mod in models:
        print(mod.id) # print model name
        #  create empty dicts
        bounds = dict() # changed model bounds
        conditions = dict()
        ovs = dict() # objective values
        f = dict() # fba fluxes
        uptake = dict() # uptake fluxes
        secretion = dict() # secretion fluxes
        rxn_info = dict()
        boundary_dicts_df= pd.DataFrame()
        boundary_labels_df= pd.DataFrame()
        
        i = 0
        ii = 0 
        
        with mod as m:
                
            # run analysis for every dict in list
            for d in boundary_dicts:

                for k in d:   # set bounds for all keys (rxnIDs in dict)
                    m.reactions.get_by_id(k).bounds = d[k] # set bounds defined in dict
                
                for rxn in rxns:
                    r = m.reactions.get_by_id(rxn)
                    m.objective = r.id
                    # run analysis for all rxns
                    fba = m.optimize() # fba
                    label = r.id
                    
                    rxn_info[i] =  [r.id,r.name,add_compartment2rxn(r)]

                    # objective values
                    ovs[i] = [m.objective, m.objective.direction, fba.objective_value]
                    print(add_compartment2rxn(r))

                    if fba.status == 'optimal':
                        try:
                            # write analysis info / results into dicts

                            # uptake / secretion
                            uptake_summary = m.summary().uptake_flux
                            secretion_summary = m.summary().secretion_flux
                            uptake[i] = pd.DataFrame(uptake_summary['flux'])
                            secretion[i] = pd.DataFrame(secretion_summary['flux'])

                            # all fluxes
                            f[label] = fba.to_frame()['fluxes']

                            pd.DataFrame(pd.DataFrame(boundary_labels).iloc[ii]).T
                            # info
                            boundary_dicts_df = pd.concat([boundary_dicts_df,pd.DataFrame(pd.DataFrame(boundary_dicts).iloc[ii]).T])
                            boundary_labels_df = pd.concat([boundary_labels_df,pd.DataFrame(pd.DataFrame(boundary_labels).iloc[ii]).T])
                        except UserWarning(...): # if no fluxes, continue
                            continue
                    i = i + 1
                ii= ii + 1

        
        # prepare dfs for excel sheet
        model_info = pd.DataFrame([m.id, m.name,m.compartments,m.annotation],\
                index=['id','name','compartments','annotation'],columns = ['model'])
        
        rxn_df = make_rxn_df(m)   
        compact_rxn_df = make_compact_rxn_df(m)  
        
        met_df = pd.DataFrame([[mi.name,mi.compartment,mi.formula,mi.charge,[r.id for r in list(mi.reactions)]] for mi in m.metabolites],\
                index=[m.id for m in m.metabolites],columns=['name','compartment','formula','charge','reactions'])
        
        rxn_info_df = pd.DataFrame(rxn_info).T.rename(\
                    columns={0:'objective rxn id',\
                            1:'objective rxn name',\
                            2:'objective reaction'})
        
        ovs_df = pd.concat([rxn_info_df,\
                            boundary_dicts_df.reset_index(),\
                            boundary_labels_df.reset_index(),\
                            pd.DataFrame(ovs, index=['objective','objective_direction','objective_value']).T]\
                        ,axis=1).drop('index', axis=1)

        
        # uptake / secretion dfs

        uptake_df = pd.concat(uptake,axis=1)
        secretion_df = pd.concat(secretion,axis=1)
        
        secretion_mets = pd.DataFrame([[met.name for met in m.reactions.get_by_id(mets).reactants] \
                                    for mets in secretion_df.index],\
                                columns=['metabolite'],index= secretion_df.index)
        uptake_mets = pd.DataFrame([[met.name for met in m.reactions.get_by_id(mets).reactants] \
                                    for mets in uptake_df.index],\
                                columns=['metabolite'],index= uptake_df.index)
        
        uptake_df = pd.merge(secretion_mets, uptake_df, left_index=True, right_index=True)  
        secretion_df = pd.merge(secretion_mets, secretion_df, left_index=True, right_index=True)  
                                    
        # sort uptake / secretion dfs
        uptake_df = uptake_df.reindex(uptake_df.fillna(0).sort_values(by=[c for c in secretion_df.columns if 'flux' in c],ascending=False).index)
        secretion_df = secretion_df.reindex(secretion_df.fillna(0).sort_values(by=[c for c in secretion_df.columns if 'flux' in c],ascending=True).index)

        
        # fluxes df
        fluxes_df = pd.concat([pd.DataFrame(f)],keys=["flux"],axis=1)
        fluxes_df = fluxes_df =pd.merge(compact_rxn_df, fluxes_df,left_index=True, right_index=True)
        # sort table on absolute flux size 
        fluxes_df= fluxes_df.reindex(fluxes_df[[c for c in fluxes_df.columns if 'flux' in c]].abs().sort_values(by=[c for c in fluxes_df.columns if 'flux' in c],ascending=False).index)
        fluxes_df[fluxes_df.index.isin([r.id for r in m.reactions if len(r.products)>0])]
        # select internal fluces only
        fluxes_df[fluxes_df.index.isin([r.id for r in m.reactions if len(r.products)>0])]
            
        # date stamp
        datestr = datetime.strftime(datetime.now(), '%H%M_%d-%m-%Y')   
        
        # write excel file
        with pd.ExcelWriter('FBA_' + results_name + '_' + datestr + '.xlsx') as writer:  
            model_info.to_excel(writer, sheet_name = 'model')
            rxn_df.to_excel(writer, sheet_name = 'reactions')
            met_df.to_excel(writer, sheet_name = 'metabolites')
            ovs_df.to_excel(writer, sheet_name = 'objective_values')  
            uptake_df.to_excel(writer, sheet_name = 'uptake')    
            secretion_df.to_excel(writer, sheet_name = 'secretion')    
            fluxes_df.to_excel(writer, sheet_name = 'fluxes')     
            
    return 

In [ ]:
from src.analysis import create_permutation_dicts
ox_labels = create_permutation_dicts({'oxygen':['anaerobic', 'limited oxygen (3.51)', 'unlimited oxygen']})
ox_bounds = create_permutation_dicts({'MAR09048_RPE':[(0,0),(-3.51,1000),(-1000,1000)]})
l_rxns = list(df['id'])
dark_new_estimates

[{'MAR09048_RPE': (0, 0)},
 {'MAR09048_RPE': (-3.51, 1000)},
 {'MAR09048_RPE': (-1000, 1000)}]

In [ ]:
fba_ATP_rxns(combined_models, ox_bounds,\
    l_rxns,\
    ox_labels,\
    results_name='TEST')

glucose[e_RPE] -->
glutamine[e_RPE] -->
alanine[e_RPE] -->
glycine[e_RPE] -->
lysine[e_RPE] -->
valine[e_RPE] -->
proline[e_RPE] -->
threonine[e_RPE] -->
serine[e_RPE] -->
leucine[e_RPE] -->
histidine[e_RPE] -->
arginine[e_RPE] -->
tyrosine[e_RPE] -->
phenylalanine[e_RPE] -->
cysteine[e_RPE] -->
tryptophan[e_RPE] -->
glutamate[e_RPE] -->
asparagine[e_RPE] -->
methionine[e_RPE] -->
aspartate[e_RPE] -->
selenomethionine[e_RPE] -->
isoleucine[e_RPE] -->
glucose[e_RPE] -->
glutamine[e_RPE] -->
alanine[e_RPE] -->
glycine[e_RPE] -->
lysine[e_RPE] -->
valine[e_RPE] -->
proline[e_RPE] -->
threonine[e_RPE] -->
serine[e_RPE] -->
leucine[e_RPE] -->
histidine[e_RPE] -->
arginine[e_RPE] -->
tyrosine[e_RPE] -->
phenylalanine[e_RPE] -->
cysteine[e_RPE] -->
tryptophan[e_RPE] -->
glutamate[e_RPE] -->
asparagine[e_RPE] -->
methionine[e_RPE] -->
aspartate[e_RPE] -->
selenomethionine[e_RPE] -->
isoleucine[e_RPE] -->
glucose[e_RPE] -->
glutamine[e_RPE] -->
alanine[e_RPE] -->
glycine[e_RPE] -->
lysine[e_RPE

C:\Users\prins\AppData\Local\Temp\ipykernel_37456\540257391.py:103: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  uptake_df = pd.merge(secretion_mets, uptake_df, left_index=True, right_index=True)
C:\Users\prins\AppData\Local\Temp\ipykernel_37456\540257391.py:104: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  secretion_df = pd.merge(secretion_mets, secretion_df, left_index=True, right_index=True)
C:\Users\prins\AppData\Local\Temp\ipykernel_37456\540257391.py:113: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  fluxes_df = fluxes_df =pd.merge(compact_rxn_df, fluxes_df,left_index=True, right_index=True)


glucose[e_RPE] -->
glutamine[e_RPE] -->


TypeError: catching classes that do not inherit from BaseException is not allowed